In [1]:
import pandas as pd
import requests
import base64
from mutagen.easyid3 import EasyID3
from mutagen.flac import FLAC
from mutagen.mp3 import MP3

In [2]:
def extract_metadata(file_path):
    try:
        if file_path.lower().endswith('.mp3'):
            audio = MP3(file_path, ID3=EasyID3)
        elif file_path.lower().endswith('.flac'):
            audio = FLAC(file_path)
        else:
            print("Unsupported file format")
            return None

        metadata = {
            'title': audio.get('title', [None])[0],
            'artist': audio.get('artist', [None])[0],
            'album': audio.get('album', [None])[0],
            'year': audio.get('date', [None])[0],
            'track_number': audio.get('tracknumber', [None])[0],
            'genre': audio.get('genre', [None])[0],
            'composer': audio.get('composer', [None])[0],
            'performer': audio.get('performer', [None])[0],
            'length': audio.info.length if hasattr(audio, 'info') else None,
            'bitrate': audio.info.bitrate if hasattr(audio, 'info') else None,
            'sample_rate': audio.info.sample_rate if hasattr(audio, 'info') else None,
            'channels': audio.info.channels if hasattr(audio, 'info') else None,
        }

        return metadata

    except Exception as e:
        print(f"Error: {e}")
        return None

# Replace song with the path to your local audio file (Test phase)
metadata = extract_metadata('Seraphine,Jasmine Clarke,Absofacto-Childhood Dreams.flac')
if metadata:
    df = pd.DataFrame([metadata])
else:
    print("No metadata extracted.")

song_title = df.loc[0, 'title']
print(song_title)
df

Childhood Dreams


,title,artist,album,year,track_number,genre,composer,performer,length,bitrate,sample_rate,channels
0,Childhood Dreams,Seraphine,Childhood Dreams,None,None,None,None,None,164.261587,869743,44100,2


In [3]:
client_id = 'f18099689fdd43ac9ced954177fcd3e6'
client_secret = 'd0930ca7a5aa4cf6a138a61d4660c165' # use your id and key
# fill in and comment out assert 
assert client_secret != 'fill your secret'
# Encode client ID and client secret
client_creds = f"{client_id}:{client_secret}"
client_creds_b64 = base64.b64encode(client_creds.encode())

token_url = "https://accounts.spotify.com/api/token"
token_data = {
    "grant_type": "client_credentials"
}
token_headers = {
    "Authorization": f"Basic {client_creds_b64.decode()}"
}

r = requests.post(token_url, data=token_data, headers=token_headers)
token_response_data = r.json()
access_token = token_response_data['access_token']

In [4]:
search_url = "https://api.spotify.com/v1/search"
search_headers = {
    "Authorization": f"Bearer {access_token}"
}
search_params = {
    "q": song_title,
    "type": "track",
    "limit": 1
}

r = requests.get(search_url, headers=search_headers, params=search_params)
search_results = r.json()
search_results
track_id = search_results['tracks']['items'][0]['id']

In [8]:
audio_features_url = f"https://api.spotify.com/v1/audio-features/{track_id}"
audio_features_headers = {
    "Authorization": f"Bearer {access_token}"
}

r = requests.get(audio_features_url, headers=audio_features_headers)
audio_features = r.json()

song_features = pd.DataFrame([audio_features])
df_reset = df.reset_index(drop=True)
df_reset

# song_features.to_csv('extracted_features_to_predict.csv',index=False)



,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.942,0.476,10,-7.778,0,0.0499,0.191,0.000487,0.0949,0.721,120.984,audio_features,0NBkXEbOvbGyUgPHVdW3k1,spotify:track:0NBkXEbOvbGyUgPHVdW3k1,https://api.spotify.com/v1/tracks/0NBkXEbOvbGy...,https://api.spotify.com/v1/audio-analysis/0NBk...,164262,4
